In [34]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [36]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200813


In [37]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [38]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_新規Vintage")

#input
input_new = "NEW_CUST_Vintage"
input_all = "NEW_CUST_Vintage_all"

input_uri = "URI_Vintage"
input_uri_all = "SUM_SHOPPING"

#out_put
output = "NEW_CUST_Vintage_all"
result = "集計_新規顧客Vintage_by_card"

output2 = "SUM_SHOPPING"
result2 = "集計_新規利用Vintage_by_card"

result3 = "集計_新規Vintage_by_card"

新規Vintage CUSTデータの読込＆集計

In [39]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv")
df = pd.read_csv(data_dir /f"{input_all}.csv")
#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')
#df = pd.concat([df1,df2])
df = df.sort_values(['cp','card','channel','mob'])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='utf-8-sig')

In [40]:
df = df[df['kanyu_yymm']>= 201803] #2018年3月以降に限定
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,cancel
508,20180331,2018,3,201803,ゴールド,オンライン,0,0,21,0.0,21,NaN
509,20180331,2018,3,201803,ゴールド,オンライン,0,1,4,0.0,4,NaN
510,20180331,2018,3,201803,ゴールド,店頭タブレット,0,0,171,0.0,171,NaN
511,20180331,2018,3,201803,ゴールド,店頭タブレット,0,1,50,0.0,50,NaN
512,20180331,2018,3,201803,ゴールド,郵送,0,0,169,0.0,169,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13280,20200531,2020,5,201805,外商,郵送,24,1,0,NaN,1030,0.0
13265,20200531,2020,5,201804,外商,郵送,25,0,0,NaN,296,8.0
13266,20200531,2020,5,201804,外商,郵送,25,1,0,NaN,575,0.0
13251,20200531,2020,5,201803,外商,郵送,26,0,0,NaN,496,3.0


In [41]:
#日付データの変換
df['cp']=pd.to_datetime(df1['cp'], format='%Y%m%d')
df['kanyu_yymmdd'] = df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format='%Y%m%d')

df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month

df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,cancel,kanyu_yymmdd,kanyu_year,kanyu_month
508,2019-03-31,2018,3,2018年3月,ゴールド,オンライン,0,0,21,0.0,21,NaN,2018-03-01,2018,3
509,2019-03-31,2018,3,2018年3月,ゴールド,オンライン,0,1,4,0.0,4,NaN,2018-03-01,2018,3
510,2019-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,0,171,0.0,171,NaN,2018-03-01,2018,3
511,2019-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,1,50,0.0,50,NaN,2018-03-01,2018,3
512,2019-03-31,2018,3,2018年3月,ゴールド,郵送,0,0,169,0.0,169,NaN,2018-03-01,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13280,2018-03-31,2020,5,2018年5月,外商,郵送,24,1,0,NaN,1030,0.0,2018-05-01,2018,5
13265,2018-03-31,2020,5,2018年4月,外商,郵送,25,0,0,NaN,296,8.0,2018-04-01,2018,4
13266,2018-03-31,2020,5,2018年4月,外商,郵送,25,1,0,NaN,575,0.0,2018-04-01,2018,4
13251,2018-03-31,2020,5,2018年3月,外商,郵送,26,0,0,NaN,496,3.0,2018-03-01,2018,3


In [42]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(5301, 15)
Int64Index([  508,   509,   510,   511,   512,   513,   514,   515,   516,
              517,
            ...
            13307, 13308, 13293, 13294, 13279, 13280, 13265, 13266, 13251,
            13252],
           dtype='int64', length=5301)
Index(['cp', 'year', 'month', 'kanyu_yymm', 'card', 'channel', 'mob', 'kzk_f',
       'no_of_new_account', 'no_of_cancel_account　', 'no_of_account', 'cancel',
       'kanyu_yymmdd', 'kanyu_year', 'kanyu_month'],
      dtype='object')
cp                       datetime64[ns]
year                              int64
month                             int64
kanyu_yymm                       object
card                             object
channel                          object
mob                               int64
kzk_f                             int64
no_of_new_account                 int64
no_of_cancel_account　           float64
no_of_account                     int64
cancel                          float64
kanyu_yymmdd             dateti

In [43]:
df['cp_date'] = df['cp'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_yymmdd'].dt.strftime('%Y/%m/%d')
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,cancel,kanyu_yymmdd,kanyu_year,kanyu_month,cp_date,kanyu_date
508,2019-03-31,2018,3,2018年3月,ゴールド,オンライン,0,0,21,0.0,21,NaN,2018-03-01,2018,3,2019/03/31,2018/03/01
509,2019-03-31,2018,3,2018年3月,ゴールド,オンライン,0,1,4,0.0,4,NaN,2018-03-01,2018,3,2019/03/31,2018/03/01
510,2019-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,0,171,0.0,171,NaN,2018-03-01,2018,3,2019/03/31,2018/03/01
511,2019-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,1,50,0.0,50,NaN,2018-03-01,2018,3,2019/03/31,2018/03/01
512,2019-03-31,2018,3,2018年3月,ゴールド,郵送,0,0,169,0.0,169,NaN,2018-03-01,2018,3,2019/03/31,2018/03/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13280,2018-03-31,2020,5,2018年5月,外商,郵送,24,1,0,NaN,1030,0.0,2018-05-01,2018,5,2018/03/31,2018/05/01
13265,2018-03-31,2020,5,2018年4月,外商,郵送,25,0,0,NaN,296,8.0,2018-04-01,2018,4,2018/03/31,2018/04/01
13266,2018-03-31,2020,5,2018年4月,外商,郵送,25,1,0,NaN,575,0.0,2018-04-01,2018,4,2018/03/31,2018/04/01
13251,2018-03-31,2020,5,2018年3月,外商,郵送,26,0,0,NaN,496,3.0,2018-03-01,2018,3,2018/03/31,2018/03/01


In [50]:
#加入月別新規数集計
q = """
        SELECT
        kanyu_yymm
        ,card
        ,sum(no_of_new_account) as sum_new_account
        from 
            df
        where 
            mob  = 0
        and card not in ("ハウスゴールド")
        group by
        kanyu_yymm,card
        """

df_new = sqldf(q, locals())
df_new

,kanyu_yymm,card,sum_new_account
0,2018年10月,ゴールド,462
1,2018年10月,一般,9712
2,2018年10月,外商,1214
3,2018年11月,ゴールド,485
4,2018年11月,一般,11210
...,...,...,...
76,2020年4月,一般,976
77,2020年4月,外商,487
78,2020年5月,ゴールド,90
79,2020年5月,一般,942


In [51]:
#加入月別口座数推移
q = """
        SELECT
        kanyu_yymm
        ,card
        ,mob
        ,sum(no_of_account) as sum_account
        ,sum(cancel) as sum_kaiyaku
        from 
            df
        where 
            card not in ("ハウスゴールド")
        group by
        kanyu_yymm,card,mob
        """

df_cust = sqldf(q, locals())
df_cust

,kanyu_yymm,card,mob,sum_account,sum_kaiyaku
0,2018年10月,ゴールド,0,461,NaN
1,2018年10月,ゴールド,1,457,NaN
2,2018年10月,ゴールド,2,455,NaN
3,2018年10月,ゴールド,3,455,NaN
4,2018年10月,ゴールド,4,455,NaN
...,...,...,...,...,...
1129,2020年4月,外商,0,487,0.0
1130,2020年4月,外商,1,487,0.0
1131,2020年5月,ゴールド,0,90,0.0
1132,2020年5月,一般,0,939,3.0


In [52]:
#マージ
q = """
        SELECT
        t1.kanyu_yymm
        ,t1.card
        ,t1.mob
        ,t1.sum_account
        ,t1.sum_kaiyaku
        ,t2.sum_new_account
        from 
            df_cust t1
        left join 
            df_new t2
        on t1.kanyu_yymm = t2.kanyu_yymm
            and t1.card = t2.card
        order by 
            t1.kanyu_yymm,t1.card,t1.mob
        """

df_cust_shukei = sqldf(q, locals())
df_cust_shukei

,kanyu_yymm,card,mob,sum_account,sum_kaiyaku,sum_new_account
0,2018年10月,ゴールド,0,461,NaN,462
1,2018年10月,ゴールド,1,457,NaN,462
2,2018年10月,ゴールド,2,455,NaN,462
3,2018年10月,ゴールド,3,455,NaN,462
4,2018年10月,ゴールド,4,455,NaN,462
...,...,...,...,...,...,...
1129,2020年4月,外商,0,487,0.0,487
1130,2020年4月,外商,1,487,0.0,487
1131,2020年5月,ゴールド,0,90,0.0,90
1132,2020年5月,一般,0,939,3.0,942


In [53]:
df_cust_shukei['残存率'] = df_cust_shukei['sum_account']/df_cust_shukei['sum_new_account']
df_cust_shukei

,kanyu_yymm,card,mob,sum_account,sum_kaiyaku,sum_new_account,残存率
0,2018年10月,ゴールド,0,461,NaN,462,0.997835
1,2018年10月,ゴールド,1,457,NaN,462,0.989177
2,2018年10月,ゴールド,2,455,NaN,462,0.984848
3,2018年10月,ゴールド,3,455,NaN,462,0.984848
4,2018年10月,ゴールド,4,455,NaN,462,0.984848
...,...,...,...,...,...,...,...
1129,2020年4月,外商,0,487,0.0,487,1.000000
1130,2020年4月,外商,1,487,0.0,487,1.000000
1131,2020年5月,ゴールド,0,90,0.0,90,1.000000
1132,2020年5月,一般,0,939,3.0,942,0.996815


In [54]:
#CSV出力
df_cust_shukei.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift=jis')

新規Vintage 売上データの読込＆集計

In [59]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_uri_all}.csv",encoding='shift-jis')
df = pd.read_csv(data_dir /f"{input_uri_all}.csv",encoding='shift-jis')
#df2 = pd.read_csv(data_dir /f"{input_uri}.csv",encoding='shift-jis')
#df = pd.concat([df1,df2])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='utf-8-sig')

In [60]:
df = df[df['kanyu_yymm']>= 201803] #2018年3月以降に限定
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt
756,20190331,2019,3,201803,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1
757,20190331,2019,3,201803,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11
758,20190331,2019,3,201803,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4
759,20190331,2019,3,201803,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1
760,20190331,2019,3,201803,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,20200531,2020,5,202005,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1
28477,20200531,2020,5,202005,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19
28478,20200531,2020,5,202005,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5
28479,20200531,2020,5,202005,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36


In [61]:
#日付データの変換
df['cp']=pd.to_datetime(df1['cp'], format='%Y%m%d')

df['kanyu_yymmdd'] = df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format='%Y%m%d')

df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month

df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt,kanyu_yymmdd,kanyu_year,kanyu_month
756,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1,2018-03-01,2018,3
757,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11,2018-03-01,2018,3
758,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4,2018-03-01,2018,3
759,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1,2018-03-01,2018,3
760,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1,2018-03-01,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,2020-05-31,2020,5,2020年5月,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1,2020-05-01,2020,5
28477,2020-05-31,2020,5,2020年5月,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19,2020-05-01,2020,5
28478,2020-05-31,2020,5,2020年5月,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5,2020-05-01,2020,5
28479,2020-05-31,2020,5,2020年5月,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36,2020-05-01,2020,5


In [62]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(9890, 18)
Int64Index([  756,   757,   758,   759,   760,   761,   762,   763,   764,
              765,
            ...
            28471, 28472, 28473, 28474, 28475, 28476, 28477, 28478, 28479,
            28480],
           dtype='int64', length=9890)
Index(['cp', 'year', 'month', 'kanyu_yymm', 'card', 'channel', 'mob', 'kzk_f',
       'hyakkaten_shoping_amt', 'gaibu_shoping_amt', 'shoping_amt',
       'hyakkaten_shopping_cnt', 'gaibu_shopping_cnt', 'shopping_cnt',
       'active_cnt', 'kanyu_yymmdd', 'kanyu_year', 'kanyu_month'],
      dtype='object')
cp                        datetime64[ns]
year                               int64
month                              int64
kanyu_yymm                        object
card                              object
channel                           object
mob                                int64
kzk_f                              int64
hyakkaten_shoping_amt              int64
gaibu_shoping_amt                float64
shoping_amt                 

In [63]:
df['cp_date'] = df['cp'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_yymmdd'].dt.strftime('%Y/%m/%d')
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt,kanyu_yymmdd,kanyu_year,kanyu_month,cp_date,kanyu_date
756,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1,2018-03-01,2018,3,2019/03/31,2018/03/01
757,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11,2018-03-01,2018,3,2019/03/31,2018/03/01
758,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4,2018-03-01,2018,3,2019/03/31,2018/03/01
759,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1,2018-03-01,2018,3,2019/03/31,2018/03/01
760,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1,2018-03-01,2018,3,2019/03/31,2018/03/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,2020-05-31,2020,5,2020年5月,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1,2020-05-01,2020,5,2020/05/31,2020/05/01
28477,2020-05-31,2020,5,2020年5月,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19,2020-05-01,2020,5,2020/05/31,2020/05/01
28478,2020-05-31,2020,5,2020年5月,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5,2020-05-01,2020,5,2020/05/31,2020/05/01
28479,2020-05-31,2020,5,2020年5月,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36,2020-05-01,2020,5,2020/05/31,2020/05/01


In [64]:
#新規数再確認
df_new

,kanyu_yymm,card,sum_new_account
0,2018年10月,ゴールド,462
1,2018年10月,一般,9712
2,2018年10月,外商,1214
3,2018年11月,ゴールド,485
4,2018年11月,一般,11210
...,...,...,...
76,2020年4月,一般,976
77,2020年4月,外商,487
78,2020年5月,ゴールド,90
79,2020年5月,一般,942


#加入月別新規数集計
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,sum(no_of_new_account) as sum_new_account
        from 
        df
        where 
        mob  = 0
        group by
        kanyu_yymm,card,channel
        """

df_new = sqldf(q, locals())
df_new

In [65]:
#加入月別売上実績
q = """
        SELECT
        kanyu_yymm
        ,card
        ,mob
        ,sum(hyakkaten_shoping_amt) as sum_hyakkaten_amt
        ,sum(gaibu_shoping_amt) as sum_gaibu_amt
        ,sum(shoping_amt) as sum_shopping_amt
        ,sum(hyakkaten_shopping_cnt) as sum_hyakkaten_trn
        ,sum(gaibu_shopping_cnt) as sum_gaibu_trn
        ,sum(shopping_cnt) as sum_shopping_trn
        ,sum(active_cnt) as sum_act_cnt
        from 
            df
        where 
            card not in ("ハウスゴールド")
        group by
            kanyu_yymm,card,mob
        """

df_uri = sqldf(q, locals())
df_uri.head()

,kanyu_yymm,card,mob,sum_hyakkaten_amt,sum_gaibu_amt,sum_shopping_amt,sum_hyakkaten_trn,sum_gaibu_trn,sum_shopping_trn,sum_act_cnt
0,2018年10月,ゴールド,0,10045850,6001841.0,16047691,844,372,1216,170
1,2018年10月,ゴールド,1,36080498,20831390.0,56911888,2595,889,3484,341
2,2018年10月,ゴールド,2,29548309,21027448.0,50575757,2825,953,3778,339
3,2018年10月,ゴールド,3,30514888,9994469.0,40509357,2753,816,3569,296
4,2018年10月,ゴールド,4,21783307,12337902.0,34121209,2349,773,3122,303


In [66]:
#マージ
q = """
        SELECT
        t1.kanyu_yymm
        ,t1.card
        ,t1.mob
        ,t1.sum_hyakkaten_amt
        ,sum(t1.sum_hyakkaten_amt) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_hyakkaten_amt
        ,t1.sum_gaibu_amt
        ,sum(t1.sum_gaibu_amt) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_gaibu_amt
        ,t1.sum_shopping_amt
        ,sum(t1.sum_shopping_amt) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_shopping_amt
        ,t1.sum_hyakkaten_trn
        ,sum(t1.sum_hyakkaten_trn) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_hyakkaten_trn
        ,t1.sum_gaibu_trn
        ,sum(t1.sum_gaibu_trn) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_gaibu_trn
        ,t1.sum_shopping_trn
        ,sum(t1.sum_shopping_trn) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_shopping_trn
        ,t1.sum_act_cnt
        ,sum(t1.sum_act_cnt) over(partition by t1.kanyu_yymm,t1.card order by mob) as cum_act_cnt
        ,t2.sum_new_account
        from 
            df_uri t1
        left join 
            df_new t2
        on t1.kanyu_yymm = t2.kanyu_yymm
            and t1.card = t2.card
        order by 
            t1.kanyu_yymm,t1.card,t1.mob
        """

df_uri_shukei = sqldf(q, locals())
df_uri_shukei

,kanyu_yymm,card,mob,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account
0,2018年10月,ゴールド,0,10045850,10045850,6001841.0,6001841.0,16047691,16047691,844,844,372,372,1216,1216,170,170,462
1,2018年10月,ゴールド,1,36080498,46126348,20831390.0,26833231.0,56911888,72959579,2595,3439,889,1261,3484,4700,341,511,462
2,2018年10月,ゴールド,2,29548309,75674657,21027448.0,47860679.0,50575757,123535336,2825,6264,953,2214,3778,8478,339,850,462
3,2018年10月,ゴールド,3,30514888,106189545,9994469.0,57855148.0,40509357,164044693,2753,9017,816,3030,3569,12047,296,1146,462
4,2018年10月,ゴールド,4,21783307,127972852,12337902.0,70193050.0,34121209,198165902,2349,11366,773,3803,3122,15169,303,1449,462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,2020年4月,外商,0,3431354,3431354,3926446.0,3926446.0,7357800,7357800,31,31,206,206,237,237,75,75,487
1130,2020年4月,外商,1,9316809,12748163,9171235.0,13097681.0,18488044,25845844,290,321,756,962,1046,1283,175,250,487
1131,2020年5月,ゴールド,0,1460177,1460177,3041764.0,3041764.0,4501941,4501941,39,39,318,318,357,357,36,36,90
1132,2020年5月,一般,0,2447627,2447627,3056091.0,3056091.0,5503718,5503718,114,114,720,720,834,834,241,241,942


In [67]:
df_uri_shukei['百貨店売上vin'] = df_uri_shukei['cum_hyakkaten_amt']/df_uri_shukei['sum_new_account']#百貨店売上Vin
df_uri_shukei['外部売上vin'] = df_uri_shukei['cum_gaibu_amt']/df_uri_shukei['sum_new_account']#外部売上Vin
df_uri_shukei['合計売上vin'] = df_uri_shukei['cum_shopping_amt']/df_uri_shukei['sum_new_account']#合計売上Vin

df_uri_shukei

,kanyu_yymm,card,mob,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account,百貨店売上vin,外部売上vin,合計売上vin
0,2018年10月,ゴールド,0,10045850,10045850,6001841.0,6001841.0,16047691,16047691,844,844,372,372,1216,1216,170,170,462,21744.264069,12990.997835,34735.261905
1,2018年10月,ゴールド,1,36080498,46126348,20831390.0,26833231.0,56911888,72959579,2595,3439,889,1261,3484,4700,341,511,462,99840.580087,58080.586580,157921.166667
2,2018年10月,ゴールド,2,29548309,75674657,21027448.0,47860679.0,50575757,123535336,2825,6264,953,2214,3778,8478,339,850,462,163797.958874,103594.543290,267392.502165
3,2018年10月,ゴールド,3,30514888,106189545,9994469.0,57855148.0,40509357,164044693,2753,9017,816,3030,3569,12047,296,1146,462,229847.500000,125227.593074,355075.093074
4,2018年10月,ゴールド,4,21783307,127972852,12337902.0,70193050.0,34121209,198165902,2349,11366,773,3803,3122,15169,303,1449,462,276997.515152,151933.008658,428930.523810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,2020年4月,外商,0,3431354,3431354,3926446.0,3926446.0,7357800,7357800,31,31,206,206,237,237,75,75,487,7045.901437,8062.517454,15108.418891
1130,2020年4月,外商,1,9316809,12748163,9171235.0,13097681.0,18488044,25845844,290,321,756,962,1046,1283,175,250,487,26176.926078,26894.622177,53071.548255
1131,2020年5月,ゴールド,0,1460177,1460177,3041764.0,3041764.0,4501941,4501941,39,39,318,318,357,357,36,36,90,16224.188889,33797.377778,50021.566667
1132,2020年5月,一般,0,2447627,2447627,3056091.0,3056091.0,5503718,5503718,114,114,720,720,834,834,241,241,942,2598.330149,3244.257962,5842.588110


In [68]:
df_uri_shukei = df_uri_shukei.sort_values(['kanyu_yymm','card','mob'])

In [69]:
#CSV出力
df_uri_shukei.to_csv(data_dir /f'{result2}.csv', header=True ,index=None, encoding='shift-jis')

In [70]:
df_merge = pd.merge(df_cust_shukei,df_uri_shukei, on =['kanyu_yymm','card','mob'], how = 'left')
df_merge

,kanyu_yymm,card,mob,sum_account,sum_kaiyaku,sum_new_account_x,残存率,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account_y,百貨店売上vin,外部売上vin,合計売上vin
0,2018年10月,ゴールド,0,461,NaN,462,0.997835,10045850,10045850,6001841.0,6001841.0,16047691,16047691,844,844,372,372,1216,1216,170,170,462,21744.264069,12990.997835,34735.261905
1,2018年10月,ゴールド,1,457,NaN,462,0.989177,36080498,46126348,20831390.0,26833231.0,56911888,72959579,2595,3439,889,1261,3484,4700,341,511,462,99840.580087,58080.586580,157921.166667
2,2018年10月,ゴールド,2,455,NaN,462,0.984848,29548309,75674657,21027448.0,47860679.0,50575757,123535336,2825,6264,953,2214,3778,8478,339,850,462,163797.958874,103594.543290,267392.502165
3,2018年10月,ゴールド,3,455,NaN,462,0.984848,30514888,106189545,9994469.0,57855148.0,40509357,164044693,2753,9017,816,3030,3569,12047,296,1146,462,229847.500000,125227.593074,355075.093074
4,2018年10月,ゴールド,4,455,NaN,462,0.984848,21783307,127972852,12337902.0,70193050.0,34121209,198165902,2349,11366,773,3803,3122,15169,303,1449,462,276997.515152,151933.008658,428930.523810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,2020年4月,外商,0,487,0.0,487,1.000000,3431354,3431354,3926446.0,3926446.0,7357800,7357800,31,31,206,206,237,237,75,75,487,7045.901437,8062.517454,15108.418891
1130,2020年4月,外商,1,487,0.0,487,1.000000,9316809,12748163,9171235.0,13097681.0,18488044,25845844,290,321,756,962,1046,1283,175,250,487,26176.926078,26894.622177,53071.548255
1131,2020年5月,ゴールド,0,90,0.0,90,1.000000,1460177,1460177,3041764.0,3041764.0,4501941,4501941,39,39,318,318,357,357,36,36,90,16224.188889,33797.377778,50021.566667
1132,2020年5月,一般,0,939,3.0,942,0.996815,2447627,2447627,3056091.0,3056091.0,5503718,5503718,114,114,720,720,834,834,241,241,942,2598.330149,3244.257962,5842.588110


In [71]:
#CSV出力
df_merge.to_csv(data_dir /f'{result3}.csv', header=True ,index=None, encoding='shift-jis')